In [35]:
import pandas as pd
import numpy as np
import os

In [6]:
import mediapipe as mp

# STEP 3: Load the input image.
image = mp.Image.create_from_file("image.jpg")

2023-09-29 19:36:27.941343: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-29 19:36:27.985035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-29 19:36:27.985079: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-29 19:36:27.985102: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-29 19:36:27.993386: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-29 19:36:27.994071: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [9]:
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [67]:
# STEP 4: Detect hand landmarks from the input image.
detection_result = detector.detect(image)
detection_result.hand_landmarks

[[NormalizedLandmark(x=0.39291912317276, y=0.7066095471382141, z=-3.293027077688748e-07, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.34312063455581665, y=0.7504479885101318, z=-0.019821790978312492, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.2708950638771057, y=0.7737331986427307, z=-0.027366425842046738, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.20807738602161407, y=0.779670238494873, z=-0.03694787248969078, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.15704940259456635, y=0.7829774022102356, z=-0.04580849036574364, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.2020859718322754, y=0.7279683947563171, z=0.004851780831813812, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.13295286893844604, y=0.743130087852478, z=-0.01850949414074421, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.09457877278327942, y=0.7587484121322632, z=-0.04270356521010399, visibility=0.0, presence=0.0),
  NormalizedLandmark(x=0.0642745643

In [91]:
# Function to recursively read files in folders
def read_images(folder_path):
    X = []
    y = []
    classes = set([])
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            
            file_name = file
            file_path = root
            
            image = mp.Image.create_from_file(file_path + "/" + file_name)
            
            detection_result = detector.detect(image)
            if(len(detection_result.hand_landmarks) == 1):
                X.append(detection_result.hand_landmarks)
                y.append(file_path[-1])
                if(file_path[-1] not in classes):
                    print(file_path[-1])
                classes.add(file_path[-1])
            
    return X, y

In [93]:
X_train, y_train = read_images("./Train_Alphabet/")

U
W
S
F
I
C
B
J
E
Y
D
L
H
A
G
Z
M
V
K
N
T
k
O
X
P
R
Q


In [94]:
X_test, y_test = read_images("./Test_Alphabet/")

U
W
S
F
I


C
B
J
E
Y
D
L
H
A
G
Z
M
V
K
N
T
O
X
P
R
Q


In [95]:
np.save("X_train", np.array(X_train))
np.save("X_test", np.array(X_test))

In [ ]:
np.save("y_train", np.array(y_train))
np.save("y_test", np.array(y_test))